# Figures for paper

- [Importing libraries](#libraries)
- [Plotting conventions](#plotting_conventions)
- [Generating figures from the full simulation](#Full_simulation_figures)<br>
    -[Density of clones in the latent reservoir early in infection](#Full_simulation_density_early_infection)<br>
    -[Decay rates after ART initiation](#decay_rates_after_ART)
- [Generating figures from the constant latent only](#constant_latent_only_figures)<br>
    -[Distribution first year on ART](#constant_latent_only_density_distribution)<br>
    -[Power law figure](#constant_latent_only_powerlaw)
- [Generating figures from the "mutation wave" simulation only](#mutation_wave_only_figures)<br>
- [Generating figures from the "Early ART" simulation only](#early_ART)<br>

    

<a class="anchor" id="libraries"></a>
## Importing libraries

In [ ]:
%matplotlib inline
from copy import deepcopy
from timeit import default_timer as timer
import math
from IPython.display import HTML

import numpy as np
import pandas as pd
import seaborn as sns

import scipy.stats as stats
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from matplotlib import animation, rc
from matplotlib import cm

from colorsys import hls_to_rgb
from brewer2mpl import sequential

In [ ]:
# Importing John's library to make figures
import mplot as mp

In [1]:
# Full library list and version numbers

print('Figures below were produced using the following packages:')

import os
import sys
from importlib import reload
print('python ==', sys.version)

import numpy as np
print('numpy ==', np.__version__)

import scipy as sp
import scipy.stats as st
print('scipy ==', sp.__version__)

import pandas as pd
print('pandas ==', pd.__version__)

import matplotlib
import matplotlib.pyplot as plt
print('matplotlib ==', matplotlib.__version__)

import seaborn as sns 
print('seaborn ==', sns.__version__)

import mplot as mp

import figures as figs

Figures below were produced using the following packages:
python == 3.10.11 | packaged by conda-forge | (main, May 10 2023, 19:01:19) [Clang 14.0.6 ]
numpy == 1.24.3
scipy == 1.10.1
pandas == 2.0.1
matplotlib == 3.7.1
seaborn == 0.12.2


In [2]:
# GLOBAL VARIABLES

DATA_DIR         = os.path.join(os.getcwd(), 'data')
DATA_DIR_FULL    = os.path.join(DATA_DIR, 'full')
DATA_DIR_EARLY   = os.path.join(DATA_DIR, 'early')
DATA_DIR_WAVE_7  = os.path.join(DATA_DIR, 'mutations_wave_7')
DATA_DIR_WAVE_10 = os.path.join(DATA_DIR, 'mutations_wave_10')
DATA_DIR_CONST_7 = os.path.join(DATA_DIR, 'constant_7')
DATA_DIR_CONST_8 = os.path.join(DATA_DIR, 'constant_8')
DATA_DIR_CONST_9 = os.path.join(DATA_DIR, 'constant_9')

<a class="anchor" id="Full_simulation_figures"></a>
# Generating figures from the full simulation
This section is to generate figures *exclusively* from the full simulation. Comparison between simulation figures, have their own separate section.

<a class="anchor" id="Full_simulation_density_early_infection"></a>
## Density of clones in the latent reservoir early in infection

In [3]:
reload(figs)

# Pass early clone information to figure generator

pdata = {
    'clone_file': os.path.join(DATA_DIR_FULL, 'clones.csv.gz'), # clone sizes
    'fig_title':  'fig-2-lr-early',                             # figure title
}

figs.plot_lr_distribution_early(**pdata)

<a class="anchor" id="decay_rates_after_ART"></a>
## Decay rates after ART initiation

In [4]:
reload(figs)

# Pass HIV-1 decay information to figure generator

pdata = {
    'total_file': os.path.join(DATA_DIR_FULL, 'totals.csv.gz'), # LR statistics
    'fig_title':  'fig-3-decay',                                # figure title
}

figs.plot_decay(**pdata)

<a class="anchor" id="constant_latent_only_figures"></a>
# Generating figures from the constant latent only
This section is to generate figures *exclusively* from the simulation where only latent clones are being tracked and produced at a constant rate. Comparison between simulation figures, have their own separate section.

<a class="anchor" id="constant_latent_only_density_distribution"></a>
## Distribution first year on ART

In [5]:
reload(figs)

# Pass post-ART clone information to figure generator

pdata = {
    'clone_file':    os.path.join(DATA_DIR_CONST_9, 'clones.csv.gz'),    # clone sizes
    'timepoints':    [0, 4, 8, 12],                                      # time points to plot
    'order':         9,                                                  # simulation size
    'mutation_file': os.path.join(DATA_DIR_WAVE_10, 'sequences.csv.gz'), # mutations
    'fig_title':     'fig-4-lr-mutations-art',                           # figure title
}

figs.plot_lr_distribution_mutations_art(**pdata)

<a class="anchor" id="constant_latent_only_powerlaw"></a>
## Power law figure

In [ ]:
def power_law_plot(df_clones, timepoints_ART, filename, order=9):
    ##timepoints_ART[0] must always be zero
    
    # PLOT FIGURE
    bmap = sequential.Blues['max']    
    ## set up figure grid

    w     = DOUBLE_COLUMN #SLIDE_WIDTH
    goldh = w / 3.2
    fig   = plt.figure(figsize=(w, goldh))

    left_bd   = 0.06
    right_bd  = 0.95
    h_space   = 0.05
    box_h     = (right_bd - left_bd - 3*h_space)/4
    top_bd    = 0.92
    bottom_bd = 0.25
    v_space   = 0.10
    box_v     = (top_bd - bottom_bd - 1*v_space)/2

    box_left   = dict(left=left_bd + 0*box_h + 0*h_space, right=left_bd + 1*box_h + 0*h_space, top=top_bd, bottom=bottom_bd)
    box_middle = dict(left=left_bd + 1*box_h + 1*h_space, right=left_bd + 2*box_h + 1*h_space, top=top_bd, bottom=bottom_bd)
    box_right  = dict(left=left_bd + 2*box_h + 2*h_space, right=left_bd + 3*box_h + 2*h_space, top=top_bd, bottom=bottom_bd)
    box_far_right  = dict(left=left_bd + 3*box_h + 3*h_space, right=left_bd + 4*box_h + 3*h_space, top=top_bd, bottom=bottom_bd)


    lineprops = {'lw': 2*SIZELINE, 'ls': '-', 'alpha': 1 }
    dashprops = {'lw': 2*SIZELINE, 'ls': ':', 'alpha': 0.5 }

    gs_left = gridspec.GridSpec(1, 1, **box_left)
    ax_left = plt.subplot(gs_left[0,0])

    gs_middle = gridspec.GridSpec(1, 1, **box_middle)
    ax_middle = plt.subplot(gs_middle[0,0])

    gs_right = gridspec.GridSpec(1, 1, **box_right)
    ax_right = plt.subplot(gs_right[0,0])

    gs_far_right = gridspec.GridSpec(1, 1, **box_far_right)
    ax_far_right = plt.subplot(gs_far_right[0,0])
    

#     pprops = { 'xlim':        [-5, 0],
#                'xticks':      [-5, -2.5, 0],
#                'xticklabels': [r'$10^{-5}$', r'$10^{-2.5}$', r'$10^{0}$'],
#                'ylim':        [0, 5],
#                'yticks':      [0, 2.5, 5],
#                'yticklabels': [r'$10^{0}$', r'$10^{2.5}$', r'$10^{5}$'],
#                'logy':        False,
#                'ylabel':      r'clone size',
#                'xlabel':      r'$p_{R}$',
#                'axoffset':    0.1,
#                'theme':       'open' }
    
    slopes = []
    
    
    for i in range(4):
        if i ==0:
            ax_used = ax_left
            pprops = { 'xlim':        [-4, 0],
               'xticks':      [-4, -2, 0],
               'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
               'ylim':        [0, 5],
               'yticks':      [0, 1, 2, 3, 4, 5],
               'yticklabels': [r'$10^{0}$', r'$10^{1}$', r'$10^{2}$', r'$10^{3}$', r'$10^{4}$', r'$10^{5}$'],
               'logy':        False,
               'ylabel':      r'Clone size (number of cells)',
               'xlabel':      r'Probability of reactivation, $p_{R}$',
               'axoffset':    0.1,
               'theme':       'open' }
            
        elif i==1:
            ax_used = ax_middle
            pprops = { 'xlim':        [-4, 0],
               'xticks':      [-4, -2, 0],
               'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
               'ylim':        [0, 5],
               'yticks':      [0, 1, 2, 3, 4, 5],
               'yticklabels': ['', '', '', '', '', ''],
               'logy':        False,
               'xlabel':      r'Probability of reactivation, $p_{R}$',
               'axoffset':    0.0,
               'theme':       'open' }
        elif i ==2:
            ax_used = ax_right
            pprops = { 'xlim':        [-4, 0],
               'xticks':      [-4, -2, 0],
               'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
               'ylim':        [0, 5],
               'yticks':      [0, 1, 2, 3, 4, 5],
               'yticklabels': ['', '', '', '', '', ''],
               'logy':        False,
               'xlabel':      r'Probability of reactivation, $p_{R}$',
               'axoffset':    -0.1,
               'theme':       'open' }
        elif i==3:
            ax_used = ax_far_right
            pprops = { 'xlim':        [-4, 0],
               'xticks':      [-4, -2, 0],
               'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
               'ylim':        [0, 5],
               'yticks':      [0, 1, 2, 3, 4, 5],
               'yticklabels': ['', '', '', '', '', ''],
               'logy':        False,
               'xlabel':      r'Probability of reactivation, $p_{R}$',
               'axoffset':    -0.2,
               'theme':       'open' }
            
            
        

#         xdat = [np.array([1,2])]
#         ydat = [np.array([-1,-2])]

#         mp.plot(type='line',ax=ax_used, x=xdat, y=ydat, colors=[BKCOLOR], plotprops=lineprops, **pprops)
        
        
        df_clones_temp = df_clones[df_clones['t']==60+timepoints_ART[i]]
        r = np.array(df_clones_temp['r'])
        L = np.array(df_clones_temp['L'])
        r = np.delete(r, np.where(L<1))
        L = np.delete(L, np.where(L<1))
        density = np.zeros((50, 50))
        for k in range(len(L)):
            i = -int((np.log10(r[k]))/0.1)  #0.1 is size of step in p_r
            j = int((np.log10(L[k]))/0.1) #0.1 is size of step in clone size
            density[i][j] += 1

        rx = np.arange(0.05,5,0.1)
        fy = np.arange(0.05,5,0.1)
        x = np.zeros(50*50)
        y = np.zeros(50*50)
        z = np.zeros(50*50)
        
        x_out = []
        y_out = []

        for i in range(50):
            x_out.append(- rx[i])
            y_largest = 0
            for j in range(50):
                if density[i][j] > 0:
                    if fy[j] > y_largest:
                        y_largest = fy[j]
            y_out.append(y_largest)
        
        for i in range(len(x_out)):
            if x_out[i] < -3.8:
                if y_out[i] < 3:
                    y_out[i] = 0
        
        x_out = list(np.array(x_out)[np.array(y_out)>0])
        y_out = list(np.array(y_out)[np.array(y_out)>0])
        
        model = LinearRegression().fit(np.array(x_out).reshape(-1, 1), np.array(y_out))
#         print(np.round(model.coef_ , 2))
        
        mp.plot(type='line',ax=ax_used, x=[x_out], y=[y_out], colors=[C_GAMMA], plotprops=lineprops, **pprops)
        
        xx = np.array(x_out)
        ax_used.plot(xx,model.predict(xx.reshape(-1,1)), '-', color = '#969696', alpha=0.5, label='Linear fit')
        
        
        
        
#         ax_used.plot(xx, yy, '.', alpha=0.5)
#         mp.plot(type='line',ax=ax_used, x=[x_out], y=[x_out], colors=[BKCOLOR], plotprops=dashprops, **pprops)
        
        slopes.append(np.round(model.coef_ , 2))
        
    ax_left.legend(loc=(0.5,0.6), prop={'size': 6}, frameon=False)
    ax_left.text(box_left['left']+0.07, box_left['top']-0.01, 'Start of ART', transform=fig.transFigure, **DEF_LABELPROPS)
    ax_left.text(box_left['left']+0.08, box_left['top']-0.05, f'm = {slopes[0][0]}', transform=fig.transFigure, **DEF_LABELPROPS)
    if timepoints_ART[1] == 1:
        ax_middle.text(box_middle['left']+0.06, box_middle['top']-0.01, '1 month on ART', transform=fig.transFigure, **DEF_LABELPROPS)
        ax_middle.text(box_middle['left']+0.08, box_middle['top']-0.05, f'm = {slopes[1][0]}', transform=fig.transFigure, **DEF_LABELPROPS)
    else:
        ax_middle.text(box_middle['left']+0.06, box_middle['top']-0.01, str(timepoints_ART[1])+' months on ART', transform=fig.transFigure, **DEF_LABELPROPS)
        ax_middle.text(box_middle['left']+0.08, box_middle['top']-0.05, f'm = {slopes[1][0]}', transform=fig.transFigure, **DEF_LABELPROPS)
    ax_right.text(box_right['left']+0.06, box_left['top']-0.01, str(timepoints_ART[2])+' months on ART', transform=fig.transFigure, **DEF_LABELPROPS)
    ax_right.text(box_right['left']+0.08, box_left['top']-0.05, f'm = {slopes[2][0]}', transform=fig.transFigure, **DEF_LABELPROPS)
    ax_far_right.text(box_far_right['left']+0.06, box_left['top']-0.01, str(timepoints_ART[3])+' months on ART', transform=fig.transFigure, **DEF_LABELPROPS)
    ax_far_right.text(box_far_right['left']+0.08, box_left['top']-0.05, f'm = {slopes[3][0]}', transform=fig.transFigure, **DEF_LABELPROPS)
    
    plt.savefig(filename , dpi = 1000, facecolor = fig.get_facecolor(), edgecolor=None, **FIGPROPS)
#     plt.close(fig)

In [ ]:
filename = 'Figures/constantLatent_firstyeardensityv2_order9_powerlaw.pdf'
power_law_plot(df_clones_constantLatent_order9, [0,4,8,12], filename, 9)

<a class="anchor" id="mutation_wave_only_figures"></a>
# Generating figures from the "mutation wave" simulation only
This section is to generate figures *exclusively* from the simulation where a constant number of new active and latent clones are being produced

In [ ]:
df_sequences_mutationwave_order10  = pd.read_csv('OutputFiles/mutations_wave/Order10/sequences.csv')

In [ ]:
# PLOT FIGURE mutation wave order 10
df = df_sequences_mutationwave_order10

lineprops = {'lw': 2*SIZELINE, 'ls': '-', 'alpha': 1 }
dashprops = {'lw': 2*SIZELINE, 'ls': ':', 'alpha': 0.5 }

pprops = { 'xlim':      [1, 7],
           'xticks':      [1, 2, 3, 4, 5, 6, 7],
           'xticklabels': [r'$1$', r'$2$', r'$3$', r'$4$', r'$5$', r'$6$', r'$7$'],
           'ylim':        [1e0, 1e5],
           'yticks':      [1e0, 10**2.5, 1e5],
           'yticklabels': [r'$10^{0}$', r'$10^{2.5}$',r'$10^{5}$'],
           'logy':        True,
           'ylabel':      'Count',
           'xlabel':      'Accumulated mutations',
           'axoffset':    0.1,
           'theme':       'open' }

## set up figure grid
    
    
w     = SINGLE_COLUMN #SLIDE_WIDTH
goldh = w / 1.1
fig   = plt.figure(figsize=(w, goldh))


# box_h   = 0.2
# box_dh  = 0.12
# box_top = 0.95

# box_left = dict(left=0.15, right=0.95, top=box_top - 0*box_h - 0*box_dh, bottom=box_top - 1*box_h - 0*box_dh)
# box_middle  = dict(left=0.15, right=0.95, top=box_top - 1*box_h - 1*box_dh, bottom=box_top - 2*box_h - 1*box_dh)
# box_right = dict(left=0.15, right=0.95, top=box_top - 2*box_h - 2*box_dh, bottom=box_top - 3*box_h - 2*box_dh)
    
    
    
w     = SINGLE_COLUMN #SLIDE_WIDTH
goldh = w / 1.1
fig   = plt.figure(figsize=(w, goldh))

left_bd   = 0.15
right_bd  = 0.95
h_space   = 0.05
top_bd    = 0.95
bottom_bd = 0.25
v_space   = 0.12
box_v     = 0.2



box_year1   = dict(left=left_bd, right=right_bd, top=top_bd, bottom=top_bd-box_v)
box_year5   = dict(left=left_bd, right=right_bd, top=bottom_bd + box_v, bottom=bottom_bd)

gs_year1 = gridspec.GridSpec(1, 1, **box_year1)
gs_year5 = gridspec.GridSpec(1, 1, **box_year5)

ax_year1 = plt.subplot(gs_year1[0,0])
ax_year5 = plt.subplot(gs_year5[0,0])


#First year

xdat = [np.array([1e3,1e4])]
ydat = [np.array([1e3,1e4])]

mp.plot(type='line',ax=ax_year1, x=xdat, y=ydat, colors=[C_DELTA], plotprops=lineprops, **pprops)

df_t = df.loc[df['t']==12]
Nmut = np.array(df_t['Nmut'])
x,t = np.unique(Nmut, return_counts = True)

ax_year1.bar(x, t, color=C_DELTA)


# Fifth year

xdat = [np.array([1e3,1e4])]
ydat = [np.array([1e3,1e4])]


mp.plot(type='line',ax=ax_year5, x=xdat, y=ydat, colors=[C_DELTA], plotprops=lineprops, **pprops)

df_t = df.loc[df['t']==60]
Nmut = np.array(df_t['Nmut'])
x,t = np.unique(Nmut, return_counts = True)

ax_year5.bar(x, t, color=C_DELTA)



# Titles 

ax_year1.text(box_year1['left']+0.3, box_year1['top']-0.01, '1 year on ART', transform=fig.transFigure, **DEF_LABELPROPS)
ax_year5.text(box_year5['left']+0.3, box_year5['top']-0.01, '5 years on ART', transform=fig.transFigure, **DEF_LABELPROPS)




# SAVE FIGURE

plt.savefig('Figures/MutationWaveOrder10.pdf' , dpi = 1000, facecolor = fig.get_facecolor(), edgecolor=None, **FIGPROPS)

<a class="anchor" id="early_ART"></a>
# Generating figures from the "Early ART" simulation only

In [ ]:
df_clones_EarlyART     = pd.read_csv('OutputFiles/EarlyART/clones.csv')

In [ ]:
##timepoints
  
timepoints = [0, 4, 8, 12]    
order = 11
    
# PLOT FIGURE
bmap = sequential.Blues['max']    
## set up figure grid

w     = DOUBLE_COLUMN #SLIDE_WIDTH
goldh = w / 4
fig   = plt.figure(figsize=(w, goldh))

left_bd   = 0.06
right_bd  = 0.97
h_space   = 0.03
box_h     = (right_bd - left_bd - 2*h_space)/4.1
top_bd    = 0.92
bottom_bd = 0.25
v_space   = 0.10
box_v     = (top_bd - bottom_bd - 1*v_space)/2

box_left   = dict(left=left_bd + 0*box_h + 0*h_space, right=left_bd + 1*box_h + 0*h_space, top=top_bd, bottom=bottom_bd)
box_middle = dict(left=left_bd + 1*box_h + 1*h_space, right=left_bd + 2*box_h + 1*h_space, top=top_bd, bottom=bottom_bd)
box_right  = dict(left=left_bd + 2*box_h + 2*h_space, right=left_bd + 3*box_h + 2*h_space, top=top_bd, bottom=bottom_bd)
box_far_right  = dict(left=left_bd + 3*box_h + 3*h_space, right=left_bd + 4*box_h + 3*h_space, top=top_bd, bottom=bottom_bd)


lineprops = {'lw': 2*SIZELINE, 'ls': '-', 'alpha': 1 }
dashprops = {'lw': 2*SIZELINE, 'ls': ':', 'alpha': 0.5 }

gs_left = gridspec.GridSpec(1, 1, **box_left)
ax_left = plt.subplot(gs_left[0,0])

gs_middle = gridspec.GridSpec(1, 1, **box_middle)
ax_middle = plt.subplot(gs_middle[0,0])

gs_right = gridspec.GridSpec(1, 1, **box_right)
ax_right = plt.subplot(gs_right[0,0])

gs_far_right = gridspec.GridSpec(1, 1, **box_far_right)
ax_far_right = plt.subplot(gs_far_right[0,0])


# pprops = { 'xlim':        [-5, 0],
#            'xticks':      [-5, -2.5, 0],
#            'xticklabels': [r'$10^{-5}$', r'$10^{-2.5}$', r'$10^{0}$'],
#            'ylim':        [0, 5],
#            'yticks':      [0, 2.5, 5],
#            'yticklabels': [r'$10^{0}$', r'$10^{2.5}$', r'$10^{5}$'],
#            'logy':        False,
#            'ylabel':      r'clone size',
#            'xlabel':      r'$p_{R}$',
#            'axoffset':    0.1,
#            'theme':       'open' }

for i in range(4):
    if i ==0:
        ax_used = ax_left
        pprops = { 'xlim':        [-4, 0],
           'xticks':      [-4, -2, 0],
           'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
           'ylim':        [0, 4],
           'yticks':      [0, 1, 2, 3, 4],
           'yticklabels': [r'$10^{0}$', r'$10^{1}$', r'$10^{2}$', r'$10^{3}$', r'$10^{4}$'],
           'logy':        False,
           'ylabel':      r'Clone size (number of cells)',
           'xlabel':      r'Probability of reactivation, $p_{R}$',
           'axoffset':    0.1,
           'theme':       'open' }
    elif i==1:
        ax_used = ax_middle
        pprops = { 'xlim':        [-4, 0],
           'xticks':      [-4, -2, 0],
           'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
           'ylim':        [0, 4],
           'yticks':      [0, 1, 2, 3, 4],
           'yticklabels': ['', '', '', '', ''],
           'logy':        False,
           'xlabel':      r'Probability of reactivation, $p_{R}$',
           'axoffset':    0.0,
           'theme':       'open' }
    elif i ==2:
        ax_used = ax_right
        pprops = { 'xlim':        [-4, 0],
           'xticks':      [-4, -2, 0],
           'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
           'ylim':        [0, 4],
           'yticks':      [0, 1, 2, 3, 4],
           'yticklabels': ['', '', '', '', ''],
           'logy':        False,
           'xlabel':      r'Probability of reactivation, $p_{R}$',
           'axoffset':    -0.1,
           'theme':       'open' }
    elif i==3:
        ax_used = ax_far_right
        pprops = { 'xlim':        [-4, 0],
           'xticks':      [-4, -2, 0],
           'xticklabels': [r'$10^{-4}$', r'$10^{-2}$', r'$10^{0}$'],
           'ylim':        [0, 4],
           'yticks':      [0, 1, 2, 3, 4],
           'yticklabels': ['', '', '', '', ''],
           'logy':        False,
           'xlabel':      r'Probability of reactivation, $p_{R}$',
           'axoffset':    -0.2,
           'theme':       'open' }

    xdat = [np.array([1,2])]
    ydat = [np.array([-1,-2])]

    mp.plot(type='line',ax=ax_used, x=xdat, y=ydat, colors=[BKCOLOR], plotprops=lineprops, **pprops)


    df_clones_temp = df_clones_EarlyART[df_clones_EarlyART['t']==timepoints[i]]
    r = np.array(df_clones_temp['r'])
    L = np.array(df_clones_temp['L'])
    r = np.delete(r, np.where(L<1))
    L = np.delete(L, np.where(L<1))
    
    ax_used.plot(np.log10(r),np.log10(L),'.')


ax_left.text(box_left['left']+0.05, box_left['top']-0.01, 'Start of Early Intervention', transform=fig.transFigure, **DEF_LABELPROPS) 
ax_middle.text(box_middle['left']+0.025, box_middle['top']-0.01, '4 months since Early Intervention', transform=fig.transFigure, **DEF_LABELPROPS)
ax_right.text(box_right['left']+0.025, box_left['top']-0.01, '8 months since Early Intervention', transform=fig.transFigure, **DEF_LABELPROPS)
ax_far_right.text(box_far_right['left']+0.025, box_left['top']-0.01, '12 months since Early Intervention', transform=fig.transFigure, **DEF_LABELPROPS)



# SAVE FIGURE

plt.savefig('Figures/earlyART.pdf' , dpi = 1000, facecolor = fig.get_facecolor(), edgecolor=None, **FIGPROPS)

In [7]:
reload(figs)

# Pass post-early intervention/elite control clone information to figure generator

pdata = {
    'clone_file':    os.path.join(DATA_DIR_EARLY, 'clones.csv.gz'),  # clone sizes
    'timepoints':    [0, 8],                                         # time points to plot
    'fig_title':     'fig-5-lr-early-intervention',                  # figure title
}

figs.plot_lr_distribution_ei(**pdata)